##### Author: Greema Regmi
##### Date : Oct 31,2022
#### This code reads the GRASP original kernel files values and creates netcdf files for each variables. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from scipy import stats
import os

import fortranformat as ff
import netCDF4

import operator
import functools
import netCDF4 as nc


# Phase funtion

In [3]:
# Reading the GRASP spherical Kernel files

def GRASP_kernel_read(Phase_read):

    filename_grid = "/home/shared/GRASP_GSFC/src/retrieval/internal_files/KERNELS_BASE/grid1.dat"

    #Reading the files
    radius_grid =[]
    with open(filename_grid) as f:
        lines_grid = f.readlines()
    for j in (lines_grid[1:42]):
        radius_grid.append(float(j))



    filename = f"/home/shared/GRASP_GSFC/src/retrieval/internal_files/KERNELS_BASE/kernels_299_{Phase_read}.txt"

    #Reading the files

    with open(filename) as f:
        lines = f.readlines()

    Real_RI =[]
    Imag_RI =[]
    index_ri =[]  #Index where the refractive indices change
    index_size =[] #Index where size parameter change
    for i in range(len(lines)-5):

        if (lines[i].split()[0] == '0.34000') == True: #This is where the refractive index change
            index_ri.append(i)
            Real_RI.append(float(lines[i].split()[1])) #Storing the values of real RI
            Imag_RI.append(float(lines[i].split()[2]))
        
        if (len(lines[i].split()) == 1) == True: #This is where the size parameter changes
            index_size.append(i)

    Real_RI = (np.unique(Real_RI)) 
    Imag_RI = -np.unique(Imag_RI)

    print(Real_RI,Imag_RI)

    ndex_ri= np.array(index_ri).reshape(22,16)
    np.unique(np.gradient(ndex_ri))

    main=[] 

    for ii in (index_ri):

        RI_1 = np.array(lines[ii+1:ii+781-1]).reshape(41,19) #The refractive index change in interval od 781 
        for s in range(41):
            P11_line=[]
            for i in range (19): P11_line.append(RI_1[s][i].split()) #Splitting the line into independendt values separated by blank space
            P_list = np.array(functools.reduce(operator.iconcat, P11_line, [])) # Concatinating Phase function values from 0-180 in one array

            P2_list = [float(j) for j in P_list] #Converting string to float
            # print(np.array(P2_list).shape)
            main.append(P2_list)

    final = np.array(main).reshape(22,16,41,181) # reshaping to (real ri,imag ri, radius,angle)


    fn = f'/home/gregmi/GRASP_scripts/GRASP_TAMU/{Phase_read}_299.nc4'
    ds = nc.Dataset(fn, 'w')

    n = ds.createDimension('RI_real', 22)
    k = ds.createDimension('RI_imag', 16)
    r = ds.createDimension('radius', 41)
    ang = ds.createDimension('S_angle', 181)



    P11 = ds.createVariable('P11', 'f4', ('RI_real', 'RI_imag', 'radius','S_angle'))
    real= ds.createVariable('n','f4',('RI_real'))
    imag= ds.createVariable('k','f4',('RI_imag'))
    angle = ds.createVariable('ang','f4',('S_angle'))
    radius = ds.createVariable('r','f4',('radius'))

    real[:] = Real_RI
    imag[:] = Imag_RI
    angle[:]= np.arange(0,181,1)
    radius[:] = radius_grid

    P11[:,:,:,:]=final
    ds.close()

    return

    

In [4]:
# Reading the GRASP spherical Kernel files

def GRASP_kernel_read(Phase_read):

    filename_grid = "/home/shared/GRASP_GSFC/src/retrieval/internal_files/KERNELS_BASE/grid1.dat"

    #Reading the files
    radius_grid =[]
    with open(filename_grid) as f:
        lines_grid = f.readlines()
    for j in (lines_grid[1:42]):
        radius_grid.append(float(j))



    filename = f"/home/shared/GRASP_GSFC/src/retrieval/internal_files/KERNELS_BASE/kernels_299_{Phase_read}.txt"

    #Reading the files

    with open(filename) as f:
        lines = f.readlines()

    Real_RI =[]
    Imag_RI =[]
    index_ri =[]  #Index where the refractive indices change
    index_size =[] #Index where size parameter change
    for i in range(len(lines)-5):

        if (lines[i].split()[0] == '0.34000') == True: #This is where the refractive index change
            index_ri.append(i)
            Real_RI.append(float(lines[i].split()[1])) #Storing the values of real RI
            Imag_RI.append(float(lines[i].split()[2]))
        
        if (len(lines[i].split()) == 1) == True: #This is where the size parameter changes
            index_size.append(i)

    Real_RI = (np.unique(Real_RI)) 
    Imag_RI = -np.unique(Imag_RI)

    # print(Real_RI,Imag_RI)

    ndex_ri= np.array(index_ri).reshape(22,16)
    np.unique(np.gradient(ndex_ri))

    main=[]  #This list stores all the values 

    for ii in (index_ri):

        RI_1 = np.array(lines[ii+1:ii+781-1]).reshape(41,19) #The refractive index change in interval od 781 
        for s in range(41):
            P11_line=[]
            for i in range (19): P11_line.append(RI_1[s][i].split()) #Splitting the line into independendt values separated by blank space
            P_list = np.array(functools.reduce(operator.iconcat, P11_line, [])) # Concatinating Phase function values from 0-180 in one array

            P2_list = [float(j) for j in P_list] #Converting string to float
            # print(np.array(P2_list).shape)
            main.append(P2_list)

    final = np.array(main).reshape(22,16,41,181) # reshaping to (real ri,imag ri, radius,angle)
    
#Creating a netcdf files for each of P11 values

    fn = f'/home/gregmi/GRASP_scripts/GRASP_TAMU/{Phase_read}_299.nc4'
    ds = nc.Dataset(fn, 'w')

    n = ds.createDimension('RI_real', 22)
    k = ds.createDimension('RI_imag', 16)
    r = ds.createDimension('radius', 41)
    ang = ds.createDimension('S_angle', 181)



    P11 = ds.createVariable('P11', 'f4', ('RI_real', 'RI_imag', 'radius','S_angle'))
    real= ds.createVariable('n','f4',('RI_real'))
    imag= ds.createVariable('k','f4',('RI_imag'))
    angle = ds.createVariable('ang','f4',('S_angle'))
    radius = ds.createVariable('r','f4',('radius'))

    real[:] = Real_RI
    imag[:] = Imag_RI
    angle[:]= np.arange(0,181,1)
    radius[:] = radius_grid

    P11[:,:,:,:]=final
    ds.close()

    return

    

In [5]:
GRASP_kernel_read(11)
GRASP_kernel_read(12)
GRASP_kernel_read(22)
GRASP_kernel_read(33)
GRASP_kernel_read(44)
GRASP_kernel_read(34)

[1.291429 1.310714 1.33     1.349286 1.368571 1.387857 1.407143 1.426429
 1.445714 1.465    1.484286 1.503571 1.522857 1.542143 1.561429 1.580714
 1.6      1.619286 1.638571 1.657857 1.677143 1.696429] [5.000000e-01 3.052701e-01 1.863797e-01 1.137923e-01 6.947477e-02
 4.241714e-02 2.589737e-02 1.581139e-02 9.653489e-03 5.893843e-03
 3.598428e-03 2.196985e-03 1.341348e-03 8.189469e-04 5.000000e-04
 1.000000e-10]


# Qext and Qabs 

In [2]:

# def GRASP_kernel_Qext(Phase_read):

filename_grid = "/home/shared/GRASP_GSFC/src/retrieval/internal_files/KERNELS_BASE/grid1.dat"

#Reading the files
radius_grid =[]
with open(filename_grid) as f:
    lines_grid = f.readlines()
for j in (lines_grid[1:42]):
    radius_grid.append(float(j))



filename = f"/home/shared/GRASP_GSFC/src/retrieval/internal_files/KERNELS_BASE/kernels_299_00.txt"

#Reading the files

with open(filename) as f:
    lines = f.readlines()

Real_RI =[]
Imag_RI =[]
index_ri =[]  #Index where the refractive indices change
index_ext =[] #Index where size parameter change
index_abs =[]
for i in range(len(lines)-5):

    if (lines[i].split()[0] == '0.34000') == True: #This is where the refractive index change
        index_ri.append(i)
        Real_RI.append(float(lines[i].split()[1])) #Storing the values of real RI
        Imag_RI.append(float(lines[i].split()[2]))
    if (lines[i].split()[0] == 'EXTINCTION') == True:
        index_ext .append(i)

    if (lines[i].split()[0] == 'ABSORPTION') == True:
        index_abs.append(i)

    
index_abs 
Real_RI = (np.unique(Real_RI)) 
Imag_RI = -np.unique(Imag_RI)

In [3]:
grad_ext = np.gradient(index_ext)
grad_abs = np.gradient(index_abs)


In [4]:
ext1=[]

for i in range (len(index_ext)):
    ex = np.array(lines[index_ext[i]+1:index_ext[i]+6])
      
    edit_space = []
    for i in range (5): edit_space.append(ex[i].split()) #Splitting the line into independendt values separated by blank space
    P_list = np.array(functools.reduce(operator.iconcat, edit_space, [])) # Concatinating Phase function values from 0-180 in one array

    P2_list = [float(j) for j in P_list] #Converting string to float
    # print(np.array(P2_list).shape)
    ext1.append(P2_list)


In [6]:
abs1=[]
for i in range (len(index_abs)):
    
    ab = np.array(lines[index_abs[i]+1:index_abs[i]+6])
        
    edit_space = []
    for i in range (5): edit_space.append(ab[i].split()) #Splitting the line into independendt values separated by blank space
    P_list = np.array(functools.reduce(operator.iconcat, edit_space, [])) # Concatinating Phase function values from 0-180 in one array

    P2_list = [float(j) for j in P_list] #Converting string to float
    # print(np.array(P2_list).shape)
    abs1.append(P2_list)

In [8]:
b_ext_vol = np.array(ext1).reshape(22,16,41)
b_abs_vol = np.array(abs1).reshape(22,16,41)


#Creating a netcdf files
fn = f'/home/gregmi/GRASP_scripts/GRASP_TAMU/299_00.nc4' #File name and path
ds = nc.Dataset(fn, 'w')

n = ds.createDimension('RI_real', 22)
k = ds.createDimension('RI_imag', 16)
r = ds.createDimension('radius', 41)



b_ext = ds.createVariable('b_ext', 'f4', ('RI_real', 'RI_imag', 'radius'))
b_abs = ds.createVariable('b_abs', 'f4', ('RI_real', 'RI_imag', 'radius'))
real= ds.createVariable('n','f4',('RI_real'))
imag= ds.createVariable('k','f4',('RI_imag'))

radius = ds.createVariable('r','f4',('radius'))

real[:] = Real_RI
imag[:] = Imag_RI

radius[:] = radius_grid

b_ext[:,:,:]=b_ext_vol
b_abs[:,:,:]=b_abs_vol
ds.close()
